In [1]:
!pip install -r requirements.txt

In [2]:
import itertools as it
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import polars as pl
from scipy import stats
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import PowerTransformer
import torch
import torch.nn as nn
from datetime import datetime
from livelossplot import PlotLosses
import random
from typing import Sequence

In [4]:
class IterativeImputerNet(nn.Module):
    def __init__(
        self,
        n_impute_features: int,
        n_time_features: int,
        n_lags: int,
        hidden_sizes: Sequence[int] = (128, 64),
        know_whats_hidden: bool = False,
        lag_list: Sequence[int] | None = None,
        impute_var_names: Sequence[str] | None = None,
        time_var_names: Sequence[str] | None = None,
        layer_type: nn.Module = nn.GELU,
    ):
        super().__init__()
        # Persist hyperparameters so checkpoints can be reconstructed and inference can run standalone.
        self.n_impute_features = n_impute_features
        self.n_time_features = n_time_features
        self.n_lags = n_lags
        self.hidden_sizes = tuple(hidden_sizes)
        self.know_whats_hidden = know_whats_hidden
        self.lags = tuple(lag_list) if lag_list is not None else tuple(range(1, n_lags + 1))
        if len(self.lags) != n_lags:
            raise ValueError("len(lag_list) must equal n_lags when provided.")
        self.output_lags = (0,) + self.lags
        self.n_output_blocks = len(self.output_lags)
        self.impute_var_names = tuple(impute_var_names) if impute_var_names else None
        self.time_var_names = tuple(time_var_names) if time_var_names else None

        self.hidden_flag_width = n_impute_features if know_whats_hidden else 0
        self.value_block_width = n_impute_features + n_time_features
        self.feature_block_width = self.value_block_width + self.hidden_flag_width
        input_dim = self.feature_block_width * (1 + n_lags)
        output_dim = n_impute_features * self.n_output_blocks  # current + lagged targets

        self.block_slices = {
            lag: slice(idx * n_impute_features, (idx + 1) * n_impute_features)
            for idx, lag in enumerate(self.output_lags)
        }

        layers = []
        in_dim = input_dim
        for h in hidden_sizes:
            layers.append(nn.Linear(in_dim, h))
            # layers.append(nn.BatchNorm1d(h))  # Add batch normalization
            if isinstance(layer_type, type):
                layers.append(layer_type())
            else:
                layers.append(layer_type)
            # layers.append(nn.Dropout(0.1))  # Add dropout for regularization
            in_dim = h
        layers.append(nn.Linear(in_dim, output_dim))
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.net(x)

    def _build_feature_matrix(
        self,
        imputed_vals: np.ndarray,
        time_vals: np.ndarray,
        mask_vals: np.ndarray | None,
    ) -> np.ndarray:
        """Construct current + lagged feature blocks, mirroring the training layout."""
        n_rows = imputed_vals.shape[0]
        features = np.full((n_rows, self.feature_block_width * (1 + self.n_lags)), np.nan, dtype=np.float64)

        if time_vals.size:
            base_block = np.concatenate([imputed_vals, time_vals], axis=1)
        else:
            base_block = imputed_vals.copy()

        features[:, :self.value_block_width] = base_block

        base_hidden = None
        if self.know_whats_hidden:
            if mask_vals is None:
                raise ValueError("mask_vals is required when know_whats_hidden=True")
            base_hidden = 1.0 - mask_vals.astype(np.float64, copy=False)
            features[:, self.value_block_width:self.feature_block_width] = base_hidden

        for idx, lag in enumerate(self.lags, start=1):
            block_start = idx * self.feature_block_width
            value_dest = features[:, block_start:block_start + self.value_block_width]
            if lag < n_rows:
                value_dest[lag:] = base_block[:-lag]
            if self.know_whats_hidden:
                hidden_dest = features[:, block_start + self.value_block_width:block_start + self.feature_block_width]
                if lag < n_rows and base_hidden is not None:
                    hidden_dest[lag:] = base_hidden[:-lag]

        return features

    @staticmethod
    def _fill_from_prev(arr: np.ndarray, curr_row_idx) -> np.ndarray:
        """Use the previous row's value if the current value is NaN."""
        if np.isnan(arr[curr_row_idx - 1]).any():
            raise ValueError(f"Cannot fill row {curr_row_idx} because previous row contains NaN values.")
        mask = np.isnan(arr[curr_row_idx])
        arr[curr_row_idx][mask] = arr[curr_row_idx - 1][mask]
        return arr

    @torch.no_grad()
    def predict(
        self,
        history_values: np.ndarray,
        curr_row_idx: int,
        mutate: bool = True,
        clip_range: tuple[float, float] | None = None,
    ) -> np.ndarray:
        # Debugging checks
        if history_values.ndim != 2:
            raise ValueError("history_values must be a 2D array.")
        n_rows, n_cols = history_values.shape
        if n_rows == 0:
            raise ValueError("history_values must contain at least one row.")
        if n_cols < self.value_block_width:
            raise ValueError(
                "history_values must include at least n_impute_features + n_time_features columns."
            )
        if curr_row_idx < 0:
            raise IndexError("curr_row_idx must be non-negative.")
        if curr_row_idx > n_rows:
            raise IndexError("curr_row_idx exceeds the number of available rows.")
        
        target_idx = curr_row_idx - 1
        if target_idx < 0:
            raise IndexError("curr_row_idx must be at least 1 to expose row 0 for inference.")

        result_array = history_values if mutate else history_values.copy()
        impute_block = result_array[:, : self.n_impute_features]
        
        target_row_vals = impute_block[target_idx]
        missing_mask = np.isnan(target_row_vals)
        if not np.any(missing_mask):
            return result_array

        if self.n_time_features:
            time_block = result_array[:, self.n_impute_features : self.value_block_width]
        else:
            time_block = np.zeros((n_rows, 0), dtype=np.float64)

        upto_idx = target_idx + 1
        values_slice = np.array(impute_block[:upto_idx], dtype=np.float64, copy=False)
        time_slice = np.array(time_block[:upto_idx], dtype=np.float64, copy=False)
        
        values_ffill = self._fill_from_prev(values_slice, len(values_slice) - 1)
        
        mask_vals = None
        if self.know_whats_hidden:
            mask_vals = (~np.isnan(values_slice)).astype(np.float64)

        features = self._build_feature_matrix(
            values_ffill,
            time_slice,
            mask_vals,
        )

        feature_row = features[target_idx]
        # Fill any NaNs in feature_row with 0 to allow imputation, should not occur
        if np.isnan(feature_row).any():
            feature_row = np.nan_to_num(feature_row, nan=0.0)

        device = next(self.parameters()).device
        feature_tensor = torch.from_numpy(feature_row).double().unsqueeze(0).to(device)
        preds = self.net(feature_tensor).cpu().numpy()
        if clip_range is not None:
            preds = np.clip(preds, clip_range[0], clip_range[1])

        current_pred = preds[0, self.block_slices[0]]
        impute_block[target_idx, missing_mask] = current_pred[missing_mask]

        return result_array

    def save_to_disk(self, path: str) -> None:
        """Serialize model weights and configuration to disk."""
        checkpoint = {
            "state_dict": self.state_dict(),
            "config": {
                "n_impute_features": self.n_impute_features,
                "n_time_features": self.n_time_features,
                "n_lags": self.n_lags,
                "hidden_sizes": self.hidden_sizes,
                "know_whats_hidden": self.know_whats_hidden,
                "lag_list": self.lags,
                "impute_var_names": self.impute_var_names,
                "time_var_names": self.time_var_names,
            },
        }
        torch.save(checkpoint, path)

    @classmethod
    def load_from_disk(cls, path: str, map_location: str | torch.device | None = None) -> "IterativeImputerNet":
        """Load a checkpoint produced by save_to_disk."""
        checkpoint = torch.load(path, map_location=map_location)
        if "config" not in checkpoint or "state_dict" not in checkpoint:
            raise ValueError("Checkpoint missing required keys 'config' and 'state_dict'.")
        model = cls(**checkpoint["config"])
        model.load_state_dict(checkpoint["state_dict"])
        return model

In [10]:
def read_dataset(location):
    read_settings = {
        "raise_if_empty": True,
        "schema_overrides": {"Time": pl.Time}
    }
    datetime = pl.datetime(
        year=pl.col("Date").dt.year(),
        month=pl.col("Date").dt.month(),
        day=pl.col("Date").dt.day(),
        hour=pl.col("Time").dt.hour(),
        minute=pl.col("Time").dt.minute(),
        second=pl.col("Time").dt.second()
    )
    return pl.read_excel(location, **read_settings).with_columns(
        pl.all().exclude("DateTime", "Date", "Time").replace(-200, None),
        DateTime=datetime,
    )

In [11]:
model = IterativeImputerNet.load_from_disk("Model A1.pth")
imputed_data = read_dataset("AirQualityUCI.xlsx").select("DateTime", "Date", "Time").hstack(pl.read_ipc("Model A1 full imputed dataset.parquet"))
original_with_still_missing_values = pl.read_ipc("missing_normalised_data.parquet")

In [ ]:
def anomaly_detection(
    original_with_still_missing_values: pl.DataFrame,
    imputed_data: pl.DataFrame,
    variable: str,
    threshold: float = 3.0
) -> pl.DataFrame:
    # Get the columns the model expects
    impute_vars = list(model.impute_var_names) if model.impute_var_names else []
    time_vars = list(model.time_var_names) if model.time_var_names else []
    model_columns = impute_vars + time_vars
    
    # Select only the columns the model uses
    imputed_np = imputed_data.select(model_columns).to_numpy().copy()
    var_idx = impute_vars.index(variable)
    
    value_differences = {}
    for i in range(len(imputed_data)):
        # Check if the variable value at row i is not missing in original_with_still_missing_values
        if original_with_still_missing_values[i, variable] is not None and not np.isnan(original_with_still_missing_values[i, variable]):
            # Store the real value
            real_val = imputed_data[i, variable]
            
            # Hide the variable value at row i
            imputed_np[i, var_idx] = np.nan
            
            # Impute the variable value at row i using the model
            model.predict(imputed_np, i + 1, mutate=True)
            
            # Get the imputed value
            imputed_val = imputed_np[i, var_idx]
            
            # Compute difference
            diff = imputed_val - real_val
            
            # Store: value_differences[DateTime] = (imputed value, real value, difference)
            datetime_val = imputed_data[i, "DateTime"]
            value_differences[datetime_val] = (imputed_val, real_val, diff)
            
            # Restore the original value back into imputed_np
            imputed_np[i, var_idx] = real_val
    
    # Compute the mean and stddev of the differences
    if not value_differences:
        return pl.DataFrame()
    
    diffs = [v[2] for v in value_differences.values()]
    mean_diff = np.mean(diffs)
    std_diff = np.std(diffs)
    
    # Return any tuples where the difference is more than threshold * stddev away from the mean
    anomalies = []
    for dt, (imp, real, diff) in value_differences.items():
        if abs(diff - mean_diff) > threshold * std_diff:
            anomalies.append({"DateTime": dt, "imputed": imp, "real": real, "difference": diff})
    
    return pl.DataFrame(anomalies)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
# starting_index = 4
# reimputed_data = (
#     imputed_df.slice(0, starting_index)
#     .vstack(
#         norm_training_data.slice(starting_index)
#     ).select(extended_variables)
# ).to_numpy()
# for i in range(starting_index, len(imputed_df)):
#     model.predict(reimputed_data, i + 1, mutate=True)